# Identifikacija zvezd

Postopek določitve višine in azimuta za vse navigacijske zvezde za določen dan,uro in položaj z uporabo navtičnih efemerid

## Kopiranje podatkov iz efemerid in njihovo parsanje

Identifikacija zvezd poteka s pomočjo uporabe navtičnih Efemeridov, kjer dobimo za posamezen dan:
1. **ime** zvezde
2. **rektascenzio** ($\text{Sha}_{\star}$) zvezde
3. **deklinacijo** ($\delta_{\star}$) zvezde
4. **časovni kot** ($\text{Sha}_{\gamma}$) pomladišča

Te podatke je nato potrebno pretvoriti v računalniško berljivo obliko, kar naredimo z naslednjimi funkcijami:
- preberemo vrstico
- parsamo iz vrstice: 
    * ime$_{\star}$,
    * $\text{Sha}_{\star}$,
    * $\delta_{\star}$
    
Nato vse te podatke vstavimo v *list* trojic [['ime','sha','dec'],['ime','sha','dec'],...,['ime','sha','dec']] in izvozimo za nadaljno uporabo in procesiranje.

In [66]:
import re # Paket za Regular Expressions

In [67]:
# convert str list ['dec','min','dec_min'] to float
def str_to_float(fs):
    
    return int(fs[0]) + (int(fs[1]) + int(fs[2])/10)/60

# read star data file and parse lines
def read_data_file(file_name):
    # opens file and reads lines
    start_file = open(file_name, 'r') 
    Lines = start_file.readlines() 

    data_list = []
    # Strips the newline character 
    for line in Lines: 
        l = line.strip() # clean newline character
        
        s_name = ''
        s_sha = 0
        s_dec = 0
        
        # parse star name
        name = re.compile('[^0-9]+')
        res = name.match(l)
        if res != None:
            s_name = res.group().strip()

        # parse star sha & dec    
        ang = re.compile('[^0-9]?[0-9]+[^0-9]?[0-9]+\.[0-9]+')
        sig_num = re.compile('[\d()-]+')
        res = ang.findall(l)
        if res != None:
            str_sha = sig_num.findall(res[0])
            str_dec = sig_num.findall(res[1])
            
            s_sha = str_to_float(str_sha)
            s_dec = str_to_float(str_dec)
            
        data_list.append([s_name, s_sha, s_dec])
    
    return data_list

In [72]:
star_data = read_data_file('star_sha_dec.txt')

[['Alpheratz', 357.6333333333333, 29.206666666666667],
 ['Ankaa', 353.17, -41.803333333333335],
 ['Schedar', 349.57166666666666, 56.653333333333336],
 ['Diphda', 348.84166666666664, -16.126666666666665],
 ['Achernar', 335.37166666666667, -56.86666666666667],
 ['Hamal', 327.9116666666667, 23.56],
 ['Polaris', 315.08666666666664, 89.35],
 ['Acamar', 315.23333333333335, -39.778333333333336],
 ['Menkar', 314.1566666666667, 4.17],
 ['Mirfak', 308.54333333333335, 49.93333333333333],
 ['Aldebaran', 290.72, 16.55],
 ['Rigel', 281.115, -7.823333333333333],
 ['Capella', 280.44166666666666, 46.016666666666666],
 ['Bellatrix', 278.43833333333333, 6.368333333333333],
 ['Elnath', 278.09833333333336, 28.623333333333335],
 ['Alnilam', 275.6816666666667, -0.8116666666666666],
 ['Betelgeuse', 270.925, 7.41],
 ['Canopus', 263.89166666666665, -51.29666666666667],
 ['Sirius', 258.48333333333335, -15.256666666666666],
 ['Adhara', 255.13833333333332, -27.003333333333334],
 ['Procyon', 244.905, 5.171666666666

Nato je potrebno te podatke pretvoriti v zvezdino višino $\text{H}_{\star}$ in njen azimut $\omega_{\star}$, kjer moramo uporabiti še časovni kot pomladišča $\text{Sha}_{\gamma}$ in naš položaj ($\varphi,\lambda$)za pretvorbo.